In [60]:
import keras
import keras_nlp

keras_nlp.models.BertTextClassifier(
    backbone, num_classes, preprocessor=None, activation=None, dropout=0.1, **kwargs
)

ModuleNotFoundError: No module named 'keras_nlp'

In [61]:
from transformers import BertTokenizer, TFBertModel
import tensorflow as tf

# Load the pre-trained BERT model and tokenizer
bert_model = TFBertModel.from_pretrained('bert-base-uncased')
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')

Some weights of the PyTorch model were not used when initializing the TF 2.0 model TFBertModel: ['cls.predictions.transform.LayerNorm.bias', 'cls.seq_relationship.bias', 'cls.predictions.bias', 'cls.predictions.transform.dense.bias', 'cls.seq_relationship.weight', 'cls.predictions.transform.dense.weight', 'cls.predictions.transform.LayerNorm.weight']
- This IS expected if you are initializing TFBertModel from a PyTorch model trained on another task or with another architecture (e.g. initializing a TFBertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing TFBertModel from a PyTorch model that you expect to be exactly identical (e.g. initializing a TFBertForSequenceClassification model from a BertForSequenceClassification model).
All the weights of TFBertModel were initialized from the PyTorch model.
If your task is similar to the task the model of the checkpoint was trained on, you can already use TFBertModel for predictions w

In [56]:
# Function to tokenize input sentences
def encode_sentences(sentences, tokenizer, max_length=128):
    # Ensure no empty or invalid sentences are present
    if not all(sentences):
        raise ValueError("Some sentences are empty or invalid.")
    
    inputs = tokenizer(
        sentences,
        max_length=max_length,
        padding='max_length',
        truncation=True,
        return_tensors='tf',
        return_attention_mask=True,
    )
    
    # Ensure that input_ids and attention_mask are returned properly
    input_ids = inputs.get('input_ids')
    attention_mask = inputs.get('attention_mask')

    if input_ids is None or attention_mask is None:
        raise ValueError("Tokenization returned None for input_ids or attention_mask.")

    return input_ids, attention_mask

# Sample sentences
sentences = ["Hello, how are you?", "I am fine, thanks."]
input_ids, attention_mask = encode_sentences(sentences, tokenizer)


In [57]:

# Define the model architecture
input_ids_layer = tf.keras.layers.Input(shape=(128,), dtype=tf.int32, name="input_ids")
attention_mask_layer = tf.keras.layers.Input(shape=(128,), dtype=tf.int32, name="attention_mask")

# BERT model
bert_output = bert_model(input_ids_layer, attention_mask=attention_mask_layer)
pooled_output = bert_output.pooler_output

# Add a classification layer
output = tf.keras.layers.Dense(1, activation='sigmoid')(pooled_output)

# Create the Keras model
model = tf.keras.Model(inputs=[input_ids_layer, attention_mask_layer], outputs=output)

# Compile the model
model.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=3e-5), loss='binary_crossentropy', metrics=['accuracy'])

# Print the input shapes for debugging
print(f"Input IDs shape: {input_ids.shape}")
print(f"Attention Mask shape: {attention_mask.shape}")

Input IDs shape: (2, 128)
Attention Mask shape: (2, 128)


In [58]:
# Dummy labels (1 for positive, 0 for negative) for binary classification
labels = [1, 0]
labels = tf.convert_to_tensor(labels)

# Train the model
model.fit([input_ids, attention_mask], labels, epochs=3, batch_size=8)

Epoch 1/3
1/1 [==============================] - 10s 10s/step - loss: 0.8081 - accuracy: 0.5000
Epoch 2/3
1/1 [==============================] - 1s 752ms/step - loss: 0.6488 - accuracy: 0.5000
Epoch 3/3
1/1 [==============================] - 1s 672ms/step - loss: 0.4834 - accuracy: 0.5000


In [54]:
test_sentences = ["This is a test sentence."]
test_input_ids, test_attention_mask = encode_sentences(test_sentences, tokenizer)

predictions = model.predict([test_input_ids, test_attention_mask])
print(predictions)

1/1 [==============================] - 1s 1s/step
[[0.34069943]]
